Вы работаете продуктовым аналитиком в компании по доставке продуктов на дом. Сервис доступен как в приложении на ios, так и на android. Вы настроили фронтовую аналитику в AppMetrica, и в конце квартала маркетинг-менеджер попросил вас проанализировать поведение пользователей, а также оценить эффективность каналов их привлечения. 

Вы выгрузили данные из AppMetrica за период с 1 января по 31 марта 2020, только по пользователям, зарегистрированным позднее 1 января 2020.

Описание данных 

- date – дата совершения события

- event - событие

    app_install – установка приложения  
    app_start – открыть приложения  
    registration – зарегистрироваться   
    search – перейти на страницу поиска товаров (каталог)  
    open_item – открыть товар  
    choose_item – отправить товар в корзину  
    tap_basket – перейти в корзину  
    purchase – подтверждение покупки  

- gender – пол пользователя

- os_name – платформа пользователя

- city – город пользователя

- device_id – идентификатор устройства пользователя

- urm_source – канал, с которого пришел пользователь

    yandex-direct – Яндекс директ  
    google_ads – реклама в Google  
    vk_ads – реклама в ВК  
    instagram_ads – реклама в instagram  
    facebook_ads – реклама в facebook  
    referral – акция «приведи друга»  
    Если стоит ‘-’, то канал не определен или это скачивание приложения напрямую или посещение не с рекламы    

- purchase_sum – стоимость покупки (при совершении события ‘purchase’)

In [1]:
import pandas as pd

In [2]:
case_data = pd.read_csv('KC_case_data .csv')
case_data.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


1. MAU февраля:

In [3]:
case_data.dtypes

date             object
event            object
purchase_sum    float64
os_name          object
device_id         int64
gender           object
city             object
utm_source       object
dtype: object

In [4]:
case_data['date'] = pd.to_datetime(case_data['date'])

In [5]:
case_data['month'] = case_data.date.dt.to_period('M')

In [6]:
case_data.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,2020-01
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,2020-01


In [7]:
case_data.groupby('month', as_index=False) \
    .agg({'device_id': 'nunique'})

,month,device_id
0,2020-01,99161
1,2020-02,75032
2,2020-03,74623


In [8]:
# 75032

2. Количество установок в январе:

In [9]:
case_data.query("event == 'app_install'") \
    .groupby('month', as_index=False) \
    .agg({'device_id': 'count'})

,month,device_id
0,2020-01,80297
1,2020-02,38078
2,2020-03,36222


In [10]:
# 80297

In [11]:
case_data = case_data.drop(columns='month')
case_data.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


3. Присвойте пользователям когорты по дню установки приложения и посчитайте для них  конверсию из установки в покупку в течение 7 дней. Для какой когорты конверсия была наибольшей? Ответ впишите в формате: дд.мм.гггг

Примечание: считаем пользователя сконвертировавшимся, если с момента установки до совершения первой покупки прошло не более 7 дней.

In [12]:
# когорты по дню установки приложения
date_app_install = case_data.query("event == 'app_install'") \
    .groupby('device_id', as_index=False) \
    .agg({'date': min}) \
    .rename(columns={'date': 'install_date'})
date_app_install

,device_id,install_date
0,4013,2020-01-15
1,4016,2020-01-04
2,4046,2020-01-04
3,4065,2020-03-19
4,4067,2020-01-31
...,...,...
154592,35379281,2020-03-29
154593,35380796,2020-03-31
154594,35381595,2020-03-30
154595,35388218,2020-03-31


In [13]:
in_purchase = case_data.merge(date_app_install, on='device_id').query("event == 'purchase'")
in_purchase

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,install_date
13,2020-01-07,purchase,269.0,android,669460,female,Moscow,-,2020-01-01
22,2020-01-09,purchase,4687.5,android,669460,female,Moscow,-,2020-01-01
50,2020-01-01,purchase,329.5,ios,833621,male,Moscow,vk_ads,2020-01-01
62,2020-01-11,purchase,601.0,ios,833621,male,Moscow,referal,2020-01-01
77,2020-01-16,purchase,277.5,ios,833621,male,Moscow,referal,2020-01-01
...,...,...,...,...,...,...,...,...,...
2262969,2020-03-31,purchase,567.5,android,21224405,female,Saint-Petersburg,yandex-direct,2020-03-31
2262979,2020-03-31,purchase,6963.0,android,11153353,female,Saint-Petersburg,-,2020-03-31
2262986,2020-03-31,purchase,165.5,android,14204302,male,Moscow,vk_ads,2020-03-31
2262993,2020-03-31,purchase,7542.0,android,18219505,female,Saint-Petersburg,-,2020-03-31


In [14]:
in_purchase['diff'] = (in_purchase.date - in_purchase.install_date).dt.days
in_purchase_7 = in_purchase.query('diff <= 7')
in_purchase_7

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,install_date,diff
13,2020-01-07,purchase,269.0,android,669460,female,Moscow,-,2020-01-01,6
50,2020-01-01,purchase,329.5,ios,833621,male,Moscow,vk_ads,2020-01-01,0
525,2020-01-01,purchase,354.5,ios,10144701,female,Moscow,-,2020-01-01,0
542,2020-01-01,purchase,597.0,android,12374612,male,Moscow,-,2020-01-01,0
569,2020-01-02,purchase,648.5,ios,12424502,female,Saint-Petersburg,instagram_ads,2020-01-01,1
...,...,...,...,...,...,...,...,...,...,...
2262969,2020-03-31,purchase,567.5,android,21224405,female,Saint-Petersburg,yandex-direct,2020-03-31,0
2262979,2020-03-31,purchase,6963.0,android,11153353,female,Saint-Petersburg,-,2020-03-31,0
2262986,2020-03-31,purchase,165.5,android,14204302,male,Moscow,vk_ads,2020-03-31,0
2262993,2020-03-31,purchase,7542.0,android,18219505,female,Saint-Petersburg,-,2020-03-31,0


In [15]:
conv = date_app_install.groupby('install_date', as_index=False) \
    .agg({'device_id': 'nunique'}) \
    .rename(columns={'device_id' : 'from_install'})
conv

,install_date,from_install
0,2020-01-01,3579
1,2020-01-02,3144
2,2020-01-03,2402
3,2020-01-04,1831
4,2020-01-05,1671
...,...,...
86,2020-03-27,1199
87,2020-03-28,1091
88,2020-03-29,1117
89,2020-03-30,994


In [16]:
conv['to_purchase'] = in_purchase_7.groupby('install_date', as_index=False) \
    .agg({'device_id': 'nunique'}).device_id 
conv

,install_date,from_install,to_purchase
0,2020-01-01,3579,1408
1,2020-01-02,3144,1186
2,2020-01-03,2402,834
3,2020-01-04,1831,639
4,2020-01-05,1671,587
...,...,...,...
86,2020-03-27,1199,277
87,2020-03-28,1091,209
88,2020-03-29,1117,223
89,2020-03-30,994,204


In [17]:
conv['CR'] = conv.to_purchase / conv.from_install *100
conv.sort_values('CR', ascending=False)

,install_date,from_install,to_purchase,CR
0,2020-01-01,3579,1408,39.340598
8,2020-01-09,1424,558,39.185393
14,2020-01-15,4310,1650,38.283063
13,2020-01-14,5173,1973,38.140344
1,2020-01-02,3144,1186,37.722646
...,...,...,...,...
77,2020-03-18,1171,235,20.068318
88,2020-03-29,1117,223,19.964190
81,2020-03-22,1261,251,19.904837
87,2020-03-28,1091,209,19.156737


In [18]:
# 01.01.2020

4. Укажите значение CR из предыдущего вопроса в % (округлите до 1 цифры после запятой)

In [19]:
conv.sort_values('CR', ascending=False)

,install_date,from_install,to_purchase,CR
0,2020-01-01,3579,1408,39.340598
8,2020-01-09,1424,558,39.185393
14,2020-01-15,4310,1650,38.283063
13,2020-01-14,5173,1973,38.140344
1,2020-01-02,3144,1186,37.722646
...,...,...,...,...
77,2020-03-18,1171,235,20.068318
88,2020-03-29,1117,223,19.964190
81,2020-03-22,1261,251,19.904837
87,2020-03-28,1091,209,19.156737


In [20]:
# 39.3

5. С какого платного маркетингового канала пришло больше всего новых пользователей?

In [21]:
case_data.groupby('utm_source', as_index=False) \
    .agg({'device_id': 'nunique'}) \
    .sort_values('device_id', ascending=False)

,utm_source,device_id
0,-,52273
6,yandex-direct,40712
2,google_ads,38096
5,vk_ads,34086
3,instagram_ads,31048
1,facebook_ads,25959
4,referal,15926


In [22]:
# Яндекс

6. Проанализируйте на каком этапе воронки отваливается бОльшая часть клиентов. Посмотрите отдельно сценарии для зарегистрированных и для незарегистрированных пользователей. На каком шаге отваливается больше всего зарегистрированных пользователей?

В ответ впишите один из шагов, конверсия из которого (в следующий шаг) самая низкая, в таком формате: «Поиск», «Добавление товара», «Переход в корзину», «Регистрация».

Подсказка: 

- Для того, чтобы выделить группу тех, кому нужно и не нужно регистрироваться, добавьте колонку с датой регистрации. Если дата регистрации < даты совершения события, то пользователь уже зарегистрирован. 

смотрим для зарегистрированных пользователей

In [39]:
case_data_registr =  case_data.query("event == 'register'") \
    .groupby('device_id', as_index=False) \
    .agg({'date': min}) \
    .rename(columns={'date':'date_registration'})
case_data_registr

,device_id,date_registration
0,4014,2020-02-01
1,4046,2020-01-04
2,4060,2020-03-07
3,4067,2020-01-31
4,4120,2020-01-07
...,...,...
78305,35342310,2020-03-27
78306,35358658,2020-03-27
78307,35367797,2020-03-29
78308,35370030,2020-03-31


In [45]:
df_registr = case_data.merge(case_data_registr, on='device_id', how='left')
df_registr

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,date_registration
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-07
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,2020-01-01
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01-01
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,NaT
...,...,...,...,...,...,...,...,...,...
2747963,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads,2020-03-31
2747964,2020-03-31,register,NaN,ios,27301864,male,Moscow,-,2020-03-31
2747965,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads,2020-03-31
2747966,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads,2020-03-31


In [52]:
funnel_registr = df_registr.query('date_registration < date') \
    .groupby('event', as_index=False) \
    .agg({'device_id': 'nunique'}) \
    .sort_values('device_id', ascending=False)
funnel_registr

,event,device_id
0,app_start,40991
3,search,40482
1,choose_item,37926
4,tap_basket,34517
2,purchase,24880


In [100]:
funnel_registr_res = funnel_registr.reset_index().drop(columns='index')
funnel_registr_res

,event,device_id
0,app_start,40991
1,search,40482
2,choose_item,37926
3,tap_basket,34517
4,purchase,24880


In [101]:
funnel_reg = []
funnel_reg = [round((funnel_registr_res.device_id[i+1] / funnel_registr_res.device_id[i])*100,1) for i in range(4)]
funnel_reg

[98.8, 93.7, 91.0, 72.1]

In [102]:
funnel_reg.insert(5,0)
funnel_reg

[98.8, 93.7, 91.0, 72.1, 0]

In [103]:
funnel_registr_res['funnel_reg'] = funnel_reg
funnel_registr_res

,event,device_id,funnel_reg
0,app_start,40991,98.8
1,search,40482,93.7
2,choose_item,37926,91.0
3,tap_basket,34517,72.1
4,purchase,24880,0.0


In [ ]:
# Переход в корзину

смотрим для всех пользователей

In [106]:
funnel_all_registr = case_data.groupby('event', as_index=False) \
    .agg({'device_id': 'nunique'}) \
    .sort_values('device_id', ascending=False)
funnel_all_registr

,event,device_id
1,app_start,190884
5,search,184488
2,choose_item,155691
0,app_install,154597
6,tap_basket,125414
4,register,78310
3,purchase,70875


In [107]:
funnel_all_registr_res = funnel_all_registr.reset_index().drop(columns='index')
funnel_all_registr_res

,event,device_id
0,app_start,190884
1,search,184488
2,choose_item,155691
3,app_install,154597
4,tap_basket,125414
5,register,78310
6,purchase,70875


In [110]:
funnel_all_reg = []
funnel_all_reg = [round((funnel_all_registr_res.device_id[i+1] / funnel_all_registr_res.device_id[i])*100,1) for i in range(6)]
funnel_all_reg

[96.6, 84.4, 99.3, 81.1, 62.4, 90.5]

In [111]:
funnel_all_reg.insert(7,0)
funnel_all_reg

[96.6, 84.4, 99.3, 81.1, 62.4, 90.5, 0]

In [112]:
funnel_all_registr_res['funnel_all_reg'] = funnel_all_reg
funnel_all_registr_res

,event,device_id,funnel_all_reg
0,app_start,190884,96.6
1,search,184488,84.4
2,choose_item,155691,99.3
3,app_install,154597,81.1
4,tap_basket,125414,62.4
5,register,78310,90.5
6,purchase,70875,0.0


In [23]:
# Переход в корзину

7. Пользователи, пришедшие с каких каналов, показали самую низкую конверсию в первую покупку?

В ответ впишите один из каналов в таком формате: «Яндекс», «Гугл», «ВК», «Фейсбук», «Инстаграм», «Реферальная программа».

Подсказка: 

- CR = кол-во юзеров, впервые совершивших покупку/кол-во юзеров, впервые открывших приложение 

In [24]:
 # кол-во юзеров, впервые совершивших покупку
case_purchase = case_data.query("event == 'purchase'") \
.groupby('utm_source', as_index=False) \
    .agg({'device_id': 'nunique'})
case_purchase

,utm_source,device_id
0,-,16598
1,facebook_ads,9017
2,google_ads,11339
3,instagram_ads,10762
4,referal,6362
5,vk_ads,12364
6,yandex-direct,12028


In [25]:
 # кол-во юзеров, впервые открывших приложение
case_app_start = case_data.query("event == 'app_start' ") \
    .groupby('utm_source', as_index=False) \
    .agg({'device_id': 'nunique'}) 
case_app_start

,utm_source,device_id
0,-,52268
1,facebook_ads,25957
2,google_ads,38092
3,instagram_ads,31045
4,referal,15924
5,vk_ads,34079
6,yandex-direct,40707


In [26]:
case_app_start['CR'] = case_purchase.device_id / case_app_start.device_id
case_app_start.sort_values('CR')

,utm_source,device_id,CR
6,yandex-direct,40707,0.295477
2,google_ads,38092,0.297674
0,-,52268,0.317556
3,instagram_ads,31045,0.346658
1,facebook_ads,25957,0.347382
5,vk_ads,34079,0.362804
4,referal,15924,0.399523


In [27]:
# Яндекс

8. Пользователи, пришедшие с какого канала, имеют медианный первый чек выше? (учитываются только первые покупки пользователей)

В ответ впишите один из каналов в таком формате: «Яндекс», «Гугл», «ВК», «Фейсбук», «Инстаграм», «Реферальная программа».

In [28]:
# выбрали покупки
# отсортировали по дате
# оставили первых, учитываются только первые покупки пользователей
case_data.query("event == 'purchase'") \
    .sort_values('date') \
    .drop_duplicates('device_id') \
    .groupby('utm_source', as_index=False) \
    .agg({'purchase_sum': 'median'}) \
    .sort_values('purchase_sum')

,utm_source,purchase_sum
1,facebook_ads,389.0
2,google_ads,390.5
6,yandex-direct,392.5
5,vk_ads,393.0
3,instagram_ads,393.5
4,referal,395.5
0,-,398.5


In [29]:
# Реферальная программа

9. Данные по затратам на рекламу:

Яндекс – 10 491 707 руб.

Гугл – 10 534 878 руб.

Фейсбук – 8 590 498 руб.

Инстаграм – 8 561626 руб.

ВК – 9 553 531руб.

Расходы на реферальную программу: если пользователь приведет друга и последний совершит первую покупку, то оба получат по 100 рублей.

Какой платный канал привлечения (среди рекламных) имеет самый высокий ROMI?   
ROMI = (Валовая прибыль − Маркетинговые расходы) / Маркетинговые расходы

В ответ впишите один из каналов в таком формате: «Яндекс», «Гугл», «ВК», «Фейсбук», «Инстаграм».

In [30]:
ROMI = case_data.groupby('utm_source', as_index=False) \
    .agg({'purchase_sum': 'sum'})
ROMI

,utm_source,purchase_sum
0,-,21449749.5
1,facebook_ads,12249901.0
2,google_ads,12868276.0
3,instagram_ads,14546969.0
4,referal,8837044.5
5,vk_ads,16389652.5
6,yandex-direct,13915368.0


In [31]:
case_data.query("event == 'purchase' and utm_source == 'referal'").device_id.nunique()

6362

In [32]:
referal_cost = case_data.query("event == 'purchase' and utm_source == 'referal'").device_id.nunique()*100*2
referal_cost

1272400

In [33]:
ROMI['market_cost'] = [0, 8590498, 10534878, 8561626, referal_cost, 9553531, 10491707]
ROMI

,utm_source,purchase_sum,market_cost
0,-,21449749.5,0
1,facebook_ads,12249901.0,8590498
2,google_ads,12868276.0,10534878
3,instagram_ads,14546969.0,8561626
4,referal,8837044.5,1272400
5,vk_ads,16389652.5,9553531
6,yandex-direct,13915368.0,10491707


In [34]:
ROMI['romi'] = (ROMI.purchase_sum - ROMI.market_cost) / ROMI.market_cost * 100
ROMI.sort_values('romi')

,utm_source,purchase_sum,market_cost,romi
2,google_ads,12868276.0,10534878,22.149265
6,yandex-direct,13915368.0,10491707,32.632068
1,facebook_ads,12249901.0,8590498,42.598264
3,instagram_ads,14546969.0,8561626,69.908952
5,vk_ads,16389652.5,9553531,71.555967
4,referal,8837044.5,1272400,594.517801
0,-,21449749.5,0,inf


In [35]:
# ВК